<a href="https://colab.research.google.com/github/MathBorgess/data_science_studies/blob/main/deep_learning/recurrent/speech_recognition/bird_spreech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [12]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d kenjee/z-by-hp-unlocked-challenge-3-signal-processing

 99% 595M/604M [00:08<00:00, 58.6MB/s]
100% 604M/604M [00:08<00:00, 75.6MB/s]


In [15]:
import zipfile
with zipfile.ZipFile("z-by-hp-unlocked-challenge-3-signal-processing.zip","r") as zip_ref:
    zip_ref.extractall("data")

In [ ]:
!pip install tensorflow-io

In [16]:

import tensorflow as tf
import tensorflow_io as tfio
import matplotlib as plt
import os

capuchin_path = os.path.join('data', 'Parsed_Capuchinbird_Clips')
not_capuchin_path = os.path.join('data', 'Parsed_Not_Capuchinbird_Clips')

In [6]:
def load_16k_mono_audio(filename):
    file_contents = tf.io.read_file(filename)
    wav, sample = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample = tf.cast(sample, tf.int64)
    return tfio.audio.resample(wav, rate_in=sample, rate_out=16000)

In [17]:
load_16k_mono_audio(os.path.join(capuchin_path, os.listdir(capuchin_path)[0]))

<tf.Tensor: shape=(80000,), dtype=float32, numpy=
array([-1.9102599e-07,  5.4285999e-07, -5.1341158e-07, ...,
        4.7466299e-03,  2.0376272e-03, -1.0874035e-03], dtype=float32)>